In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [2]:
#make sql connection to the table
#apply filteration steps
#then fetch one image and display
import math
import random
import warnings
import sqlite3
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [3]:
import pandas as pd

#train_table=pd.read_csv("/Users/mv96/Desktop/temp/post_doc_eng/train.csv")
test_table=pd.read_csv("/Volumes/My_Book/Theoremkb/cluster_package/8-finetuning_data/resources/test_1.csv")
val_table=pd.read_csv("/Volumes/My_Book/Theoremkb/cluster_package/8-finetuning_data/resources/val_1.csv")

print(test_table.shape,val_table.shape)

(2354445, 7) (2356075, 7)


In [4]:
pdfs=list(val_table["pdf_path"].unique())
print(len(pdfs))

12515


In [5]:
#size

In [6]:
val_table

,label,pdf_path,page_no,top_left,bot_right,global_fonts,text
0,0,1905.12372/refFst-arxiv.pdf,1,"(200.03563410896913, 1533.5334869875485)","(1453.814536340852, 2044.3130581444573)",['cmr12 cmr12 cmr12 cmr12 cmr12 cmr12 cmr12 cm...,Proving lower bounds on the size of propositio...
1,0,1905.12372/refFst-arxiv.pdf,2,"(200.03563410896913, 210.58577869761257)","(1453.814536340852, 520.6711844589736)",['cmr12 cmr12 cmr12 cmmi12 cmr12 cmr12 cmr12 c...,assignment of formula x’. In a propositional f...
2,0,1905.12372/refFst-arxiv.pdf,2,"(200.03563410896913, 531.7020004166135)","(1453.814536340852, 934.701886183888)",['cmr12 cmr12 cmr12 cmr12 cmr12 cmr12 cmr12 cm...,Pudlák [8] found connections between the refle...
3,0,1905.12372/refFst-arxiv.pdf,2,"(200.03563410896913, 1103.6095189458335)","(1453.814536340852, 1327.0599520222552)",['cmr12 cmr12 cmr12 cmr12 cmr12 cmr12 cmr12 cm...,The result that requires the most work is the ...
4,3,1905.12372/refFst-arxiv.pdf,2,"(200.03563410896913, 1357.5127839859158)","(1453.814536340852, 1794.3831096835756)","['cmr12 cmr12 cmmi8 ', 'cmmi8 cmr12 cmr12 cmr1...","Here REFF s,t, missing the lower index denotin..."
...,...,...,...,...,...,...,...
2356070,0,1210.7831/AdcockHansenShadrin3.2.pdf,14,"(250.0, 1609.9722222222222)","(1448.611111111111, 1638.9166666666665)",['cmr10 cmr10 cmr9 cmr9 cmr9 cmr9 cmr9 cmr9 cm...,Figure 3: Errors for the polynomial least squa...
2356071,0,1210.7831/AdcockHansenShadrin3.2.pdf,14,"(250.0, 1646.9999999999998)","(1450.0, 1729.7222222222224)",['cmr9 cmmi9 cmr9 cmr9 cmr9 cmmi9 cmmi9 cmmi9 ...,"against m = 10, 20, . . . , 200. Squares corre..."
2356072,0,1210.7831/AdcockHansenShadrin3.2.pdf,15,"(250.0, 258.77777777777777)","(1450.0277777777776, 482.30555555555554)",['cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cm...,Notice also one interesting facet of Figure 3:...
2356073,0,1210.7831/AdcockHansenShadrin3.2.pdf,15,"(250.0, 491.22222222222223)","(1450.0, 648.3611111111111)",['cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cm...,"To summarize, Fourier extensions appear to pre..."


In [7]:
print(val_table["page_no"].iloc[0])

1


In [8]:
from ast import literal_eval

val_table['page_no'] = val_table["page_no"].apply(lambda x: literal_eval(str(x)))
val_table['top_left'] = val_table["top_left"].apply(lambda x: literal_eval(str(x)))
val_table['bot_right'] = val_table["bot_right"].apply(lambda x: literal_eval(str(x)))

test_table['page_no'] = test_table["page_no"].apply(lambda x: literal_eval(str(x)))
test_table['top_left'] = test_table["top_left"].apply(lambda x: literal_eval(str(x)))
test_table['bot_right'] = test_table["bot_right"].apply(lambda x: literal_eval(str(x)))

In [9]:
#print(train_table.shape) #has to be 6 columned

print(val_table.shape)
print(test_table.shape)

(2356075, 7)
(2354445, 7)


In [10]:
test_table

,label,pdf_path,page_no,top_left,bot_right,global_fonts,text
0,0,1003.3473/ProjectiveFINAL.pdf,1,"(351.8611111111111, 1056.1944444444446)","(1348.138888888889, 1512.1944444444443)",['cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cm...,This paper provides the foundations of a surpr...
1,0,1003.3473/ProjectiveFINAL.pdf,1,"(351.8611111111111, 1521.111111111111)","(1348.1388888888887, 1611.8055555555554)",['cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cm...,The definitions that support expressions such ...
2,0,1003.3473/ProjectiveFINAL.pdf,1,"(351.8611111111111, 1620.75)","(1348.138888888889, 1943.8888888888887)",['cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cm...,Iterated function systems are a standard frame...
3,0,1003.3473/ProjectiveFINAL.pdf,2,"(351.8611111111111, 331.13888888888886)","(1348.111111111111, 388.6111111111111)",['cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cm...,"objects such as arcs of circles and parabolas,..."
4,0,1003.3473/ProjectiveFINAL.pdf,2,"(351.8611111111111, 397.55555555555554)","(1348.138888888889, 587.8611111111111)",['cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cmr10 cm...,The behavior of attractors of projective IFSs ...
...,...,...,...,...,...,...,...
2354440,1,1212.1626/Reduccion20131015.pdf,6,"(861.8333333333333, 1622.2222222222222)","(1328.4166666666665, 1693.8333333333333)","['cmr10 cmmi10 ', 'cmmi10 ', 'cmmi10 ', 'cmr10...","(0, t), ∂f ∂s (0, t) U 0 (t), t ∈ I − {t i } n..."
2354441,1,1212.1626/Reduccion20131015.pdf,6,"(444.6111111111111, 1877.583333333333)","(1270.9444444444443, 1949.1666666666665)",['cmmi10 cmsy8 cmr10 cmr10 cmmi10 cmsy8 cmr10 ...,"g′(t) =Z′(t), W0(t)=R ∂f ∂t (0, t), ∂f ∂s (0, ..."
2354442,1,1212.1626/Reduccion20131015.pdf,7,"(351.8611111111111, 327.30555555555554)","(1348.1111111111113, 391.88888888888886)",['cmr10 cmmi10 cmr10 cmr10 cmr10 cmr10 cmmi8 c...,"Since g(t) is continuous on [t i , t i+1 ], i ..."
2354443,1,1212.1626/Reduccion20131015.pdf,7,"(963.7499999999999, 830.8055555555554)","(1155.6388888888887, 862.3611111111111)",['cmr10 cmmi10 cmmi10 cmr8 cmr10 cmmi10 cmr8 c...,"(0, t) U 0 (t), W 0 (t) dt"


In [1]:
sub_sample_table="./subsample_for_vision.csv"

import pandas as pd
sample_df=pd.read_csv(sub_sample_table)

from ast import literal_eval
sample_df['page_no'] = sample_df["page_no"].apply(lambda x: literal_eval(str(x)))
sample_df['top_left'] = sample_df["top_left"].apply(lambda x: literal_eval(str(x)))
sample_df['bot_right'] = sample_df["bot_right"].apply(lambda x: literal_eval(str(x)))


In [2]:
print(sample_df.shape)
sample_df=sample_df.dropna()
print(sample_df.shape)

(232443, 7)
(232443, 7)


In [67]:
def extract_and_save_image(row,df,folder_name,image_location,ext=".png"):
    try:
        label=row[1]["label"]
        local_folder=image_location+"/"+folder_name+"/label_"+str(label)
        if(not os.path.exists(local_folder)):
            os.mkdir(local_folder)
        image_path=row[1]["pdf_path"].rsplit("/",1)[0]
        sub_folder=image_path.split(".")[0]
        generic_path=image_path+"/images"+"/image_{}.png".format(str(row[1]["page_no"]))
        file_name=sub_folder+"/"+generic_path
        full_path=os.path.join(results_directory,file_name)
        img_file_name=str(generic_path.rsplit("/",2)[0])+"_"+str(row[1]["page_no"])+"_"+str(row[0])+ext
        full_name=os.path.join(local_folder,img_file_name)
        if(os.path.exists(full_name)):
            return 1
        img = cv2.imread(full_path) #add 0 for grey scale
        top_left=math.floor(row[1]["top_left"][0]),math.floor(row[1]["top_left"][1])
        bot_right=math.floor(row[1]["bot_right"][0]),math.floor(row[1]["bot_right"][1])
        cropped_image = img[top_left[1]:bot_right[1], top_left[0]:bot_right[0]]
        status=cv2.imwrite(full_name, cropped_image)
    except TypeError or FileExistsError:
        return
    except:
        print("error--",row[1]["pdf_path"])
        return


In [17]:
image_location="/Users/mv96/Downloads/vision_patches" #path to the dir where you will store the patches this should exist before
folder_name="val_data"
results_directory="/Volumes/My_Book/Theoremkb/cluster_package/5-grobid_on_pdfs/ftest" #the directory where you saved data csvs images etc
import time
from tqdm import tqdm
tick=time.time()

for row in tqdm(val_table.iterrows()):
    if(not os.path.exists(os.path.join(image_location,folder_name))):
        os.mkdir(os.path.join(image_location,folder_name))
    df=val_table
    folder_name="val_data"
    extract_and_save_image(row,df,folder_name,image_location,ext=".png")
    
tock=time.time()
print(tock-tick)


0it [00:00, ?it/s]
1it [00:00,  3.15it/s]
4it [00:00, 11.36it/s]
8it [00:00, 18.19it/s]
12it [00:00, 22.82it/s]
16it [00:00, 25.93it/s]
19it [00:00, 26.28it/s]
23it [00:01, 27.45it/s]
26it [00:01, 27.79it/s]
29it [00:01, 27.52it/s]
33it [00:01, 28.52it/s]
37it [00:01, 29.40it/s]
41it [00:01, 30.43it/s]
45it [00:01, 31.07it/s]
49it [00:01, 31.69it/s]
53it [00:02, 32.18it/s]
57it [00:02, 32.58it/s]
61it [00:02, 32.06it/s]
65it [00:02, 31.92it/s]
69it [00:02, 30.37it/s]
73it [00:02, 30.53it/s]
77it [00:02, 29.90it/s]
81it [00:02, 30.04it/s]
85it [00:03, 30.10it/s]
89it [00:03, 29.77it/s]
92it [00:03, 29.61it/s]
96it [00:03, 30.01it/s]
100it [00:03, 30.01it/s]
104it [00:03, 30.30it/s]
108it [00:03, 30.82it/s]
112it [00:03, 31.15it/s]
116it [00:04, 30.76it/s]
120it [00:04, 30.92it/s]
124it [00:04, 31.05it/s]
128it [00:04, 30.47it/s]
132it [00:04, 30.17it/s]
136it [00:04, 30.20it/s]
140it [00:04, 30.45it/s]
144it [00:04, 30.77it/s]
148it [00:05, 30.68it/s]
152it [00:05, 30.62it/s]
156it [00

KeyboardInterrupt: 

In [22]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed
from tqdm import tqdm

image_location="/Users/mv96/Downloads/vision_patches"
results_directory="/Volumes/My_Book/Theoremkb/cluster_package/5-grobid_on_pdfs/ftest" #the directory where you saved data csvs images etc
n_jobs=-2

for df,folder_name in zip([sample_df],["sub_sample_data"]):
    if(not os.path.exists(os.path.join(image_location,folder_name))):
        os.mkdir(os.path.join(image_location,folder_name))
    res=Parallel(n_jobs=n_jobs,backend="threading",verbose=2)(delayed(extract_and_save_image)
                                           (row,df,folder_name,image_location) for row in tqdm(df.iterrows()))




0it [00:00, ?it/s][Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.



1it [00:00,  7.11it/s][Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-2)]: Done 351 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-2)]: Done 634 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-2)]: Done 999 tasks      | elapsed:    0.2s



1155it [00:00, 5815.05it/s][Parallel(n_jobs=-2)]: Done 1444 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-2)]: Done 1971 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-2)]: Done 2578 tasks      | elapsed:    0.3s



2618it [00:00, 9496.64it/s][Parallel(n_jobs=-2)]: Done 3267 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-2)]: Done 4036 tasks      | elapsed:    0.4s



4193it [00:00, 11826.57it/s][Parallel(n_jobs=-2)]: Done 4887 tasks      | elapsed:    0.5s



5757it [00:00, 13153.69it/s][Parallel(n_jobs=-2)]: Done 5818 tasks      | elapsed:    0.5s


error-error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_2_187093.png
 /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_2_187092.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_3_187100.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_4_187109.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_5_187122.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_6_187137.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_6_187138.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_8_187160.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_8_187161.png





187199it [00:12, 5976.02it/s] 

error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_9_187172.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_9_187173.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_10_187187.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_10_187188.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_11_187199.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_11_187200.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_11_187201.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_12_187216.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_12_187217.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_12_187218.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_13_1

[Parallel(n_jobs=-2)]: Done 187258 tasks      | elapsed:   13.0s


error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_16_187276.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_18_187297.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_18_187298.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_19_187310.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_20_187324.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_20_187325.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_20_187326.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_21_187337.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_21_187339.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_21_187338.png
error- /Users/mv96/Downloads/vision_patches/sub_sample_data/label_0/1110.3751_22




188466it [00:20, 574.33it/s] 


189358it [00:27, 367.06it/s]


189987it [00:31, 291.14it/s]


190431it [00:34, 261.68it/s]


190747it [00:36, 240.89it/s]


190973it [00:37, 226.55it/s]


191136it [00:38, 218.55it/s]


191255it [00:39, 209.65it/s]


191343it [00:39, 203.15it/s]


191410it [00:40, 192.39it/s]


191461it [00:40, 189.26it/s]


191501it [00:41, 182.88it/s]


191534it [00:41, 170.07it/s]


191560it [00:41, 173.10it/s]


191585it [00:41, 167.20it/s]


191606it [00:41, 165.90it/s]


191626it [00:42, 161.06it/s]


191644it [00:42, 163.60it/s]


191662it [00:42, 156.41it/s]


191681it [00:42, 152.20it/s]


191702it [00:42, 148.97it/s]


191718it [00:42, 150.53it/s]


191737it [00:42, 144.26it/s]


191758it [00:42, 153.52it/s]


191786it [00:43, 175.11it/s]


191814it [00:43, 188.86it/s]


191842it [00:43, 204.66it/s]


191870it [00:43, 209.04it/s]


191892it [00:43, 211.24it/s]


191914it [00:43, 212.42it/s]


191940it [00:43, 215.65it/s]


191968it [00:43, 224.47it/s]


1919

In [53]:
from ast import literal_eval
#freeze the base model
def setting_up_labels(label):
    label=label.lower()
    if("basic" in label):
        return "basic"
    if("overlap" in label):
        return "overlap"
    if("proof" in label):
        return "proof"
    if("theorem" in label):
        return  "theorem"
    
def setting_up_labels_1(label):
    label=label.lower()
    if("basic" in label):
        return 0
    if("overlap" in label):
        return 3
    if("proof" in label):
        return 1
    if("theorem" in label):
        return  2
    

def preprocess_df_for_vision(df):
    #string conversion
    df['page_no'] = df["page_no"].apply(lambda x: literal_eval(str(x)))
    df['top_left'] = df["top_left"].apply(lambda x: literal_eval(str(x)))
    df['bot_right'] = df["bot_right"].apply(lambda x: literal_eval(str(x)))
    df["label"]=df["label"].apply(setting_up_labels)
    #dropping nans same as in NLP approach
    df=df.dropna()
    df["label"]=df["label"].apply(setting_up_labels_1)
    
    return df

#order of execution
names=['batch_28.csv',
 'batch_18.csv',
 'batch_108.csv',
 'batch_106.csv',
 'batch_85.csv',
 'batch_123.csv',
 'batch_7.csv',
 'batch_48.csv',
 'batch_2.csv',
 'batch_114.csv',
 'batch_125.csv',
 'batch_67.csv',
 'batch_4.csv']

In [61]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed
from tqdm import tqdm
import glob

image_location="/Users/mv96/Downloads/vision_patches"
results_directory="/Volumes/My_Book/Theoremkb/cluster_package/5-grobid_on_pdfs/ftest" #the directory where you saved data csvs images etc
n_jobs=-2

train_data_dir="/Volumes/My_Book/Theoremkb/cluster_package/8-finetuning_data/train_data"
csv_files=glob.glob(train_data_dir+"/**.csv")

order=[]
for name in names:
    for file in csv_files:
        if(name in file):
            order.append(file)

In [70]:
for file in tqdm(order):
    folder_name=file[:-4].rsplit("/")[-1]
    df=pd.read_csv(file)
    df=preprocess_df_for_vision(df)
    print(folder_name,df.shape)
    
    #continue if the zip file or folder they exist
    zip_path=os.path.join(image_location,folder_name)+".zip"
    if(os.path.exists(zip_path)):
        continue
    if(not os.path.exists(os.path.join(image_location,folder_name))):
        os.mkdir(os.path.join(image_location,folder_name))
    else:
        continue
    res=Parallel(n_jobs=n_jobs,backend="threading",verbose=2)(delayed(extract_and_save_image)
                                           (row,df,folder_name,image_location) for row in tqdm(df.iterrows()))
    


  0%|                                                    | 0/13 [00:00<?, ?it/s]/var/folders/tm/9klk1nzn2lz690wvdfb85s_00000gn/T/ipykernel_2037/534099145.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"]=df["label"].apply(setting_up_labels_1)
  8%|███▍                                        | 1/13 [00:08<01:39,  8.27s/it]

batch_28 (172585, 26)


/var/folders/tm/9klk1nzn2lz690wvdfb85s_00000gn/T/ipykernel_2037/534099145.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"]=df["label"].apply(setting_up_labels_1)


batch_18 (180245, 26)




0it [00:00, ?it/s][Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.




...

batch_108 (174950, 26)




0it [00:00, ?it/s][Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.




error--error-- 1310.4672/hinftycalculussimilarity_nocomments.pdf
 1310.4672/hinftycalculussimilarity_nocomments.pdf


21it [00:00, 105.95it/s]

35it [00:00, 118.84it/s][Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    0.3s


56it [00:00, 124.32it/s]

77it [00:00, 132.82it/s]

91it [00:00, 133.65it/s]

105it [00:00, 135.14it/s]

119it [00:00, 117.21it/s]

140it [00:01, 127.94it/s]

154it [00:01, 129.07it/s][Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:    1.3s


175it [00:01, 138.07it/s]

196it [00:01, 145.60it/s]

211it [00:01, 146.44it/s]

231it [00:01, 145.96it/s]

246it [00:01, 146.39it/s]

261it [00:01, 138.84it/s]

280it [00:02, 145.07it/s]

301it [00:02, 151.74it/s]

322it [00:02, 148.95it/s]

343it [00:02, 151.10it/s][Parallel(n_jobs=-2)]: Done 351 tasks      | elapsed:    2.6s


364it [00:02, 152.89it/s]

385it [00:02, 155.78it/s]

406it [00:02, 160.37it/s]

427it [00:02, 160.37it/s]

448it [00:03, 161.16it/s]

469it [00:03, 162.19it/s]

486it [00:03, 142.62it/s]

504it [00:03, 135.93it/s]

518it [00:03, 136.86it/s]

539it [00:03, 139.81it/s]

554it [00:03, 142.24it/s]

574it [00

KeyboardInterrupt: 

In [4]:
import os
image_location="/Volumes/MV96/vision_patches"
check_dir=os.path.join(image_location,"test_data")
print(check_dir)
counts=0
for label in os.listdir(check_dir):
    counts+=len(os.listdir(os.path.join(check_dir,label)))
    #print(train_table.shape)
print(counts)    

/Volumes/MV96/vision_patches/test_data
76986
